# Importing Required Libraries


In [1]:
import pandas as pd
import numpy as np

# Loading the required Data

In [2]:
# Load Data 
df = pd.read_csv("Case_Study_for_Data_Test_25.csv", encoding="latin1")
print(df.head())


     ID  age     car_type  Loanamount    Deposit    area   APR  \
0  9997   61  Convertible   £24,108.00    £881.00  rural  0.15   
1  9996   42       Saloon    £5,988.00    £967.00  rural  0.20   
2  9995   57       Saloon    £5,099.00    £973.00  urban  0.15   
3  9994   32  Convertible   £20,774.00  £1,478.00  rural   NaN   
4  9993   29       Saloon    £4,612.00    £774.00  urban   NaN   

  application_outcome Funded   
0            approved      No  
1            approved      No  
2            approved      No  
3            declined     NaN  
4            declined     NaN  


# Initial Data Exploration

In [3]:
# Display Shape of the DataFrame
print(f"\nShape of the DataFrame: {df.shape}")

# Display Columns in the DataFrame
print(f"\nColumns: {df.columns.tolist()}")

# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Display Info about the DataFrame
print("\nDataFrame Info:")
print(df.info())



Shape of the DataFrame: (10000, 9)

Columns: ['ID', 'age', 'car_type', ' Loanamount ', ' Deposit ', 'area', 'APR', 'application_outcome', 'Funded ']
Number of duplicate rows: 0

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   car_type             10000 non-null  object 
 3    Loanamount          10000 non-null  object 
 4    Deposit             10000 non-null  object 
 5   area                 10000 non-null  object 
 6   APR                  6470 non-null   float64
 7   application_outcome  10000 non-null  object 
 8   Funded               6470 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 703.3+ KB
None


# Standardize Column names

In [4]:
# Rename columns to remove spaces, '_' and convert to lowercase
df.columns = [c.strip().lower().replace("_", "") for c in df.columns]
print(f"\nColumns: {df.columns.tolist()}")


Columns: ['id', 'age', 'cartype', 'loanamount', 'deposit', 'area', 'apr', 'applicationoutcome', 'funded']


# Preprocess loanamount & deposit

In [5]:
# Remove £ and Convert to Numeric Columns
for col in ["loanamount", "deposit"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace("[£$,]", "", regex=True)
        .str.strip()
        .replace("", np.nan)
        .astype(float)
    )
print(df.head())

     id  age      cartype  loanamount  deposit   area   apr  \
0  9997   61  Convertible     24108.0    881.0  rural  0.15   
1  9996   42       Saloon      5988.0    967.0  rural  0.20   
2  9995   57       Saloon      5099.0    973.0  urban  0.15   
3  9994   32  Convertible     20774.0   1478.0  rural   NaN   
4  9993   29       Saloon      4612.0    774.0  urban   NaN   

  applicationoutcome funded  
0           approved     No  
1           approved     No  
2           approved     No  
3           declined    NaN  
4           declined    NaN  


# Preprocess applicationoutcome & funded

In [6]:
# Encoding categories approved/declined to 1/0 and yes/no to 1/0
df["applicationoutcome"] = df["applicationoutcome"].str.strip().str.lower().map({"approved": 1, "declined": 0})
df["funded"] = df["funded"].str.strip().str.lower().map({"yes": 1, "no": 0})
print(df.head())


     id  age      cartype  loanamount  deposit   area   apr  \
0  9997   61  Convertible     24108.0    881.0  rural  0.15   
1  9996   42       Saloon      5988.0    967.0  rural  0.20   
2  9995   57       Saloon      5099.0    973.0  urban  0.15   
3  9994   32  Convertible     20774.0   1478.0  rural   NaN   
4  9993   29       Saloon      4612.0    774.0  urban   NaN   

   applicationoutcome  funded  
0                   1     0.0  
1                   1     0.0  
2                   1     0.0  
3                   0     NaN  
4                   0     NaN  


# Impute Missing Values

In [7]:
# Fill missing values with 0 for apr and funded
df = df.fillna({'apr': 0})
df["funded"] = df["funded"].fillna(0).astype(int)
print(df.head())

     id  age      cartype  loanamount  deposit   area   apr  \
0  9997   61  Convertible     24108.0    881.0  rural  0.15   
1  9996   42       Saloon      5988.0    967.0  rural  0.20   
2  9995   57       Saloon      5099.0    973.0  urban  0.15   
3  9994   32  Convertible     20774.0   1478.0  rural  0.00   
4  9993   29       Saloon      4612.0    774.0  urban  0.00   

   applicationoutcome  funded  
0                   1       0  
1                   1       0  
2                   1       0  
3                   0       0  
4                   0       0  


# Create Additional Features

In [8]:
# 1. Age buckets
age_bins = [0, 24, 34, 44, 54, 65]
age_labels = ['<25', '25-34', '35-44', '45-54', '55+']
df['age_bucket'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=True)

# 2. Deposit-to-loan ratio
df['depositratio'] = df['deposit'] / (df['loanamount'] + 1e-9)  # epsilon to avoid divide-by-zero

# 3. APR percentage buckets
apr_bins = [0, 5, 10, 15, 20, 25, np.inf]
apr_labels = ['0%','5%', '10%', '15%', '20%', '25%']
df['apr_pct'] = pd.cut(df['apr']*100, bins=apr_bins, labels=apr_labels, right=False)

print(df.head())

     id  age      cartype  loanamount  deposit   area   apr  \
0  9997   61  Convertible     24108.0    881.0  rural  0.15   
1  9996   42       Saloon      5988.0    967.0  rural  0.20   
2  9995   57       Saloon      5099.0    973.0  urban  0.15   
3  9994   32  Convertible     20774.0   1478.0  rural  0.00   
4  9993   29       Saloon      4612.0    774.0  urban  0.00   

   applicationoutcome  funded age_bucket  depositratio apr_pct  
0                   1       0        55+      0.036544     15%  
1                   1       0      35-44      0.161490     20%  
2                   1       0        55+      0.190822     15%  
3                   0       0      25-34      0.071147      0%  
4                   0       0      25-34      0.167823      0%  


# Describe preprocessed clean data

In [9]:
print("\nDataframe Description:")
df.describe(include='all')


Dataframe Description:


,id,age,cartype,loanamount,deposit,area,apr,applicationoutcome,funded,age_bucket,depositratio,apr_pct
count,10000.00000,10000.000000,10000,10000.000000,10000.000000,10000,10000.00000,10000.000000,10000.000000,10000,10000.000000,10000
unique,NaN,NaN,3,NaN,NaN,2,NaN,NaN,NaN,5,NaN,6
top,NaN,NaN,SUV,NaN,NaN,rural,NaN,NaN,NaN,25-34,NaN,0%
freq,NaN,NaN,3830,NaN,NaN,5290,NaN,NaN,NaN,2640,NaN,3530
mean,5000.50000,40.477400,NaN,11920.430200,2668.142100,NaN,0.09628,0.647000,0.306900,NaN,0.266187,NaN
std,2886.89568,14.015903,NaN,5882.825283,1669.184029,NaN,0.08784,0.477927,0.461231,NaN,0.192726,NaN
min,1.00000,18.000000,NaN,2502.000000,500.000000,NaN,0.00000,0.000000,0.000000,NaN,0.020134,NaN
25%,2500.75000,27.000000,NaN,7527.250000,1106.000000,NaN,0.00000,0.000000,0.000000,NaN,0.112256,NaN
50%,5000.50000,40.000000,NaN,11218.500000,2364.500000,NaN,0.10000,1.000000,0.000000,NaN,0.220400,NaN
75%,7500.25000,53.000000,NaN,15922.750000,4079.250000,NaN,0.15000,1.000000,1.000000,NaN,0.354381,NaN


# Saving the cleaned data

In [10]:
df.to_csv("cleaned_loan_data.csv", index=False)